In [1]:
import sys

sys.path.append('/usr/local/lib/python3.8/site-packages')

from os import path as Path
import ast
from owlready2 import *
import javalang.tree

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
ontology_path = './computed/tree.owl'
chess_path = './resources/android-chess/app/src/main/java/jwtc/chess/'
treePy_path = './resources/tree.py'
ontology_populated_path = './computed/tree2.owl'

In [3]:
class Visitor(ast.NodeVisitor):

    def __init__(self, ontology):
        self.ontology = ontology

    def generic_visit(self, node):
        ast.NodeVisitor.generic_visit(self, node)
        with self.ontology as onto:
            if type(node) == ast.ClassDef:
                for obj in node.bases:
                    if obj.id == "Node":
                        types.new_class(node.name, (Thing,))
                    else:
                        types.new_class(node.name, (onto[obj.id],))

            elif type(node) == ast.Assign:
                for el in node.value.elts:
                    if el.s == "body" or el.s == "parameters":
                        types.new_class(el.s, (ObjectProperty,))
                    else:
                        types.new_class(
                            "jname" if el.s == "name" else el.s,
                            (DataProperty,))


In [4]:
def create_ontology(path):
    with open(path, "r") as source:
        tree = ast.parse(source.read())

    ontology = get_ontology("http://Java_Ontology/tree.owl")

    visitor = Visitor(ontology)
    visitor.visit(tree)
    
    ontology.save(ontology_path, format="rdfxml")

    print('Ontology created ==> "%s" ' % ontology_path)
    
create_ontology("./resources/tree.py")

Ontology created ==> "./computed/tree.owl" 


In [5]:
def populate_ontology(onto, source):
    for file in os.listdir(source):
        if file.endswith('.java'):
            f_path = os.path.join(source, file)
            with open(f_path) as j_file:
                with onto:
                    ast = javalang.parse.parse(j_file.read())
                    for _, node in ast.filter(javalang.tree.ClassDeclaration):
                        class_declaration = onto['ClassDeclaration']()
                        class_declaration.jname = [node.name]
                        for member in node.body:
                            if type(member) == javalang.tree.FieldDeclaration:
                                    append_fields(class_declaration, member, onto)
                            elif type(member) == javalang.tree.MethodDeclaration or type(member) == javalang.tree.ConstructorDeclaration:
                                    append_method(class_declaration, member, onto)
    onto.save(ontology_populated_path, format="rdfxml")
    print('done')

def append_fields(class_declaration, field, onto):
    for fld in field.declarators:
        fd = onto['FieldDeclaration']()
        fd.jname = [fld.name]
        class_declaration.body.append(fd)

def append_method(class_declaration, method, onto):
    md = onto[method.__class__.__name__]()
    md.jname = [method.name]
    append_statements(md, method, onto)
    append_parameters(md, method, onto)
    class_declaration.body.append(md)

def append_statements(md, method, onto):
    for _, statement in method.filter(javalang.tree.Statement):
        if type(statement) != javalang.tree.Statement:
            s_type = statement.__class__.__name__
            s = onto[s_type]()
            md.body.append(s)

def append_parameters(md, method, onto):
    for _, statement in method.parameters:
        fp = onto['FormalParameter']()
        md.parameters.append(fp)



    

In [6]:
#  TODO :: delete ontology tree2 before recreating
# os.remove(ontology_populated_path)


onto = get_ontology(ontology_path).load()
populate_ontology(onto, chess_path)

done


In [ ]:
onto = get_ontology(ontology_path).load()
cd = onto["ClassDeclaration"]

assert cd.name == "ClassDeclaration"
assert len(cd.is_a) == 1
assert cd.is_a[0].name == 'TypeDeclaration'